In [1]:
#Only good for static websites
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
#https://ece.ucdavis.edu/undergraduate/undergraduate-advising

url = "https://ece.ucdavis.edu/undergraduate/undergraduate-advising"
# base_url = "https://catalog.ucdavis.edu"

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [ ]:
#All the links and names associated with the bubble icons on the site

links = soup.find_all('div', class_ = 'layout__region priority-links')
titles_scraped = soup.find_all('div', class_ = 'vertical-link__title')
titles = []

for title in titles_scraped:
    stripped = str(title).split('>')[1]
    if 'New Students' not in str(stripped).replace('&amp;', "").replace('</div', ''):
        if str(stripped).replace('&amp;', "").replace('</div', '').replace('\n', '').replace('  ', '') == 'DegreesRequirements':
            titles.append("Degree Requirements")
        else:
            titles.append(str(stripped).replace('&amp;', "").replace('</div', '').replace('\n', '').replace('  ', ''))


list_of_scraped_links = {'Advising for New Students': 'https://ece.ucdavis.edu/undergraduate/aggie-advising',
                         'IDP Program': 'https://ece.ucdavis.edu/undergraduate/integrated-degree-programs',
                         'College of Engineering': 'https://engineering.ucdavis.edu/undergraduates/academic-advising'}

i = 0
for scrape in links:
    for a in scrape.find_all('a', href=True):
        if "https" in a['href']:
            # print("Found the URL:", a['href'])
            list_of_scraped_links[titles[i]] = a['href']
            i = i + 1
    
    # print(len(list_of_scraped_links))


list_of_scraped_links['Change of Major'] = 'https://ece.ucdavis.edu/undergraduate/frequently-asked-questions'



print(list_of_scraped_links)


{'Advising for New Students': 'https://ece.ucdavis.edu/undergraduate/aggie-advising', 'IDP Program': 'https://ece.ucdavis.edu/undergraduate/integrated-degree-programs', 'College of Engineering': 'https://engineering.ucdavis.edu/undergraduates/academic-advising', 'Degree Requirements': 'https://ece.ucdavis.edu/undergraduate/degrees-requirements', 'Course Offerings': 'https://ece.ucdavis.edu/course-schedules', 'Mandatory Academic Advising': 'https://ece.sf.ucdavis.edu/undergraduate/annual-mandatory-advising', 'Release Holds': 'https://ece.sf.ucdavis.edu/undergraduate/annual-mandatory-advising', 'Internship Info.Forms': 'https://ece.sf.ucdavis.edu/undergraduate/forms', 'PTA Request': 'https://ece.ucdavis.edu/pta-request-1', "FAQ's": 'https://ece.ucdavis.edu/undergraduate/frequently-asked-questions', 'Change of Major': 'https://ece.ucdavis.edu/undergraduate/frequently-asked-questions'}


In [171]:
def check_info(item):
    #Email: "mailto:eceugradinfo@ucdavis.edu">eceugradinfo@ucdavis.edu</a><span>   </span>

    if 'mailto' in item:
        cleaned = item.replace('Email:', '').replace('\"mailto:', '').replace('\">eceugradinfo@ucdavis', '').replace('</a><span>', '').strip()
        cleaned = str(cleaned.split('.edu')[0]) + '.edu'
        return cleaned
    elif 'Click' in item:
        cleaned = item.replace('Click "', '')

        return cleaned.split('.edu')[0] + '.edu'
    elif 'Office' in item:
        return item.replace('Office Location: ', '')
    
    elif " " == item:
        pass
    else:
        return item

In [184]:
#This is a database for the advisors and their info
#name, office location, email, Advises, Office hours (in person), office hours(remote), link to schedule appointment
councelor_to_info = {}
councelors_page = soup.find('div', class_ = 'block block-layout-builder block-inline-blocksf-basic panel')
councelor_buffer = councelors_page.find_all('div')
councelor_more_buffer = councelor_buffer[0]
list_ = councelor_more_buffer.find_all('p')
list2 = councelor_more_buffer.find_all('li')

list_of_needed_componenets = []
list2 = list2[2:]
list2 = list2[:-6]
cleaned_up = []

for li in list2:
    cleaned_up.append(str(li).replace('<li>', '').replace('</li>', '').replace('<span>', '').replace('</span>', ''))

# print(*cleaned_up, sep='\n')
    

for l in list_:
    text = str(l).replace('<p>', '').replace('</p>', '').replace('<a href=', '')+ '\n'
    list_of_needed_componenets.append(text)

new_list = list_of_needed_componenets[5:]
del new_list[5:7]
del new_list[5]
del new_list[10:14]


# print(*new_list, sep='\n')
# print(len(new_list))

info = []
#councelor_to_info
i = 0
k=0
name = ""
for stuff in new_list:
    if i == 0 or i == 5 or i == 10:
        name = new_list[i]
        councelor_to_info[name] = []
    else:
        item = check_info(stuff)
        if 'Advising' in item:
            # print('here')
            for j in range(3):
                if k == 5:
                    councelor_to_info[name].append(cleaned_up[k])
                    break
                else:
                    councelor_to_info[name].append(cleaned_up[k])
                k = k + 1
        else:
            councelor_to_info[name].append(item)
    
    i = i + 1


print(councelor_to_info)















{'Ileana Oseguera\n': ['2064 Kemper\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0\xa0\n', 'eceugradinfo@ucdavis.edu', 'Students with last names: A-L', 'In-Person: Tuesday-Thursday:\xa09:00 am\xa0- 4:00 pm (by appointment)', 'Remote: Monday, Friday:\xa09:00 am - 4:00 pm (by appointment)', 'https://appointments.ucdavis.edu'], 'Pascal Gonzalez\n': ['2064 Kemper\n', 'eceugradinfo@ucdavis.edu', 'Students with last names: M-Z', 'In-Person: Mondays:\xa09:00 am - 4:00 pm (by appointment)', 'Remote: Tuesday-Wednesday:\xa09:00 am - 4:00 pm (by appointment)', 'https://appointments.ucdavis.edu'], 'Hussain Al-Asaad\n': ['eceugradinfo@ucdavis.edu', 'Website:\xa0"https://ece.ucdavis.edu/directory/hussain-al-asaad">https://www.ece.ucdavis.edu/~halasaad/</a>\n', 'Office hours:\xa0\n', '\xa0\n', '<strong>Engineering Undergraduate Office</strong>\n', 'Drop-in hours:\xa0"https://engineering.ucdavis.edu/undergraduates/academ